In [6]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns

In [7]:
# import dataset
fraud = pd.read_csv("D:/DATA SCIENCE/1. Data Scientist/Assignment/14.Random Forests/Fraud_check.csv")

In [8]:
fraud.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [9]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [10]:
fraud.shape

(600, 6)

In [11]:
fraud.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [12]:
#treating those who have taxable_income <= 30000 as "Risky" and others are "Good"
#Converting the Taxable income variable to bucketing[] .
fraud["income"]="<=30000"
fraud.loc[fraud["Taxable.Income"]>=30000,"income"]="Good"
fraud.loc[fraud["Taxable.Income"]<=30000,"income"]="Risky"

In [13]:
#Drop variable =  "Taxable.Income"
fraud.drop(["Taxable.Income"],axis=1,inplace=True)

In [14]:
# encoding
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for column_name in fraud.columns:
    if fraud[column_name].dtype == object:
        fraud[column_name] = le.fit_transform(fraud[column_name])
    else:
        pass

In [15]:
fraud.head()

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,income
0,0,2,50047,10,1,0
1,1,0,134075,18,1,0
2,0,1,160205,30,1,0
3,1,2,193264,15,1,0
4,0,1,27533,28,0,0


In [16]:
#Splitting the data into x_feature() and y_labels
x_features = fraud.iloc[:,0:5]
y_labels = fraud.iloc[:,5]

In [17]:
# Collecting the column names
colnames = list(fraud.columns)
predictors = colnames[0:5]
target = colnames[5]

In [29]:
# here Splitting the data into train and test by using stratifyb sampling
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_features,y_labels,test_size = 0.3,stratify = y_labels)

In [30]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((420, 5), (420,), (180, 5), (180,))

In [31]:
#Model building
from sklearn.ensemble import RandomForestClassifier as RF
model = RF(n_jobs = 3,n_estimators = 15, oob_score = True, criterion = "entropy")
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=15, n_jobs=3,
                       oob_score=True)

In [32]:
model.estimators_

[DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=917618744),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=83761368),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=361035695),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=228739597),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=834552106),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=1451075201),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=1599209996),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
                        random_state=2057431690),
 DecisionTreeClassifier(criterion='entropy', max_features='auto',
            

In [33]:
model.classes_

array([0, 1])

In [34]:
model.n_features_

5

In [35]:
model.n_outputs_

1

In [36]:
model.n_classes_

2

In [37]:
model.oob_score_

0.719047619047619

In [38]:
# Prediction on training data
pred_train = model.predict(x_train)

In [39]:
#Accuracy on training data
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train,pred_train)

In [40]:
np.mean(pred_train == y_train)

0.9857142857142858

In [41]:
# creating Confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_train,pred_train)

In [42]:
# Prediction on testing data
pred_test = model.predict(x_test)

In [43]:
#Accuracy on testing data
accuracy_test =accuracy_score(y_test,pred_test)

In [45]:
np.mean(pred_test == y_test)

0.7722222222222223

In [46]:
##Confusion matrix
con_test = confusion_matrix(y_test,pred_test)

In [47]:
con_test

array([[136,   7],
       [ 34,   3]], dtype=int64)

In [48]:
# plot a Decision tree present in Random forest
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
conda install -c conda-forge pydotplus

In [49]:
import pydotplus
from io import StringIO

In [50]:
tree = model.estimators_[6]

In [51]:
dot_data = StringIO()
export_graphviz(tree,out_file = dot_data, filled = True,rounded = True, feature_names = predictors ,class_names = target,impurity =False)

In [52]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [53]:
#  for selected decision tree Creating pdf and png file 
graph.write_pdf('fraudrf.pdf')
graph.write_png('fraudrf.png')

True